In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [3]:
import numpy as np
import gymnasium as gym
from gymnasium import spaces
from typing import Optional, Dict, Any, Tuple
import os
import time

from stable_baselines3 import DQN
from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.utils import set_random_seed
from stable_baselines3.common.logger import configure

# =============================================================================
# 1. THE (p0, z, theta) ENVIRONMENT
# =============================================================================

class SearchEnvVec(gym.Env):
    """
    A custom Gymnasium environment for the two-object search problem.

    This version uses a flat state representation based on:
    1. The search count vector (z_t)
    2. The static prior (p_0)
    3. The related object status (theta)
    """

    def __init__(self, p0: np.ndarray, gamma1: np.ndarray, gamma2: np.ndarray, T: int, cost_vector: np.ndarray):
        super(SearchEnvVec, self).__init__()

        self.n = p0.shape[0]
        self.T = T
        self.p0 = p0.astype(np.float32)
        self.p0_flat = self.p0.flatten()
        self.gamma1 = gamma1.astype(np.float32)
        self.gamma2 = gamma2.astype(np.float32)
        self.cost_vector = cost_vector.astype(np.float32)

        self.action_space = spaces.Discrete(self.n)

        # --- Define the observation space ---
        # 1. z_vector: n elements
        # 2. prior: n*n elements
        # 3. theta: n+1 elements (one-hot)
        obs_size = self.n + (self.n * self.n) + (self.n + 1)

        self.observation_space = spaces.Box(
            low=0.0,
            high=float(self.T),
            shape=(obs_size,),
            dtype=np.float32
        )

        self._precompute_conditionals()

        # Environment state variables
        self.z_vector = np.zeros(self.n, dtype=np.int32)
        self.theta = 0
        self.current_step = 0
        self.true_pos_o1 = 0
        self.true_pos_o2 = 0

    def _precompute_conditionals(self):
        self.conditionals = np.zeros_like(self.p0)
        for j in range(self.n):
            col_sum = np.sum(self.p0[:, j])
            if col_sum > 0:
                self.conditionals[:, j] = self.p0[:, j] / col_sum
            else:
                self.conditionals[:, j] = 1.0 / self.n

    def _get_obs(self) -> np.ndarray:
        """
        Constructs the observation vector from z_t, p_0, and theta.
        """
        z_flat = self.z_vector.astype(np.float32)
        theta_one_hot = np.zeros(self.n + 1, dtype=np.float32)
        theta_one_hot[self.theta] = 1.0

        # Concatenate all parts into a single flat vector
        obs = np.concatenate([z_flat, self.p0_flat, theta_one_hot])
        return obs

    def reset(self, seed: Optional[int] = None, options: Optional[Dict] = None) -> Tuple[np.ndarray, Dict]:
        super().reset(seed=seed)

        self.z_vector = np.zeros(self.n, dtype=np.int32)
        self.theta = 0
        self.current_step = 0

        p_flat = self.p0.flatten()
        # Handle case where prior is all zeros by adding a small epsilon or checking sum
        prior_sum = np.sum(p_flat)
        if prior_sum == 0:
             true_idx = self.np_random.choice(self.n * self.n) # Choose randomly if prior is zero
        else:
             true_idx = self.np_random.choice(self.n * self.n, p=p_flat / prior_sum) # Normalize

        self.true_pos_o1, self.true_pos_o2 = np.unravel_index(true_idx, (self.n, self.n))

        info = {}
        return self._get_obs(), info

    def step(self, action: int) -> Tuple[np.ndarray, float, bool, bool, Dict]:
        if not self.action_space.contains(action):
            raise ValueError(f"Invalid action: {action}")

        self.current_step += 1

        reward = -float(self.cost_vector[action])
        terminated = False
        found_o1 = False
        found_o2_this_step = False

        # --- Check for O1 (Target) Detection ---
        # O1 is present if its true location is the action cell AND
        # (O2 is hidden AND O1 is at action cell) OR (O2 is found at its true location AND O1 is at action cell)
        is_o1_present = (action == self.true_pos_o1)

        if is_o1_present:
            if self.np_random.random() > self.gamma1[action]:
                found_o1 = True

        if found_o1:
            reward = 1.0
            terminated = True
        else:
            # --- Check for O2 (Related) Detection ---
            if self.theta == 0: # Only check for O2 if it's currently hidden
                if action == self.true_pos_o2:
                    if self.np_random.random() > self.gamma2[action]:
                        self.theta = action + 1 # O2 found, update theta

            self.z_vector[action] += 1 # Increment search count for the actioned cell


        if self.current_step >= self.T:
            terminated = True

        info = {'found_o1': found_o1, 'found_o2': found_o2_this_step}
        truncated = False # We use terminated for end of horizon
        return self._get_obs(), reward, terminated, truncated, info

# =============================================================================
# 2. DQN TRAINING FUNCTION FOR A SINGLE RUN
# =============================================================================

def run_single_experiment(run_id: int, base_log_dir: str, seed: int, total_timesteps: int) -> Dict[str, float]:
    """
    Runs a single DQN training and evaluation experiment.
    Returns a dictionary of final evaluation metrics.
    """
    print(f"\n===== STARTING DQN RUN {run_id} with SEED {seed} ====")

    # --- 3.1. Problem and Environment Definition ---
    NUM_CELLS = 5
    TIME_HORIZON = 10

    set_random_seed(seed)

    prior = np.array([
        [0.152,  0.0039, 0.003,  0.0108, 0.011],
        [0.0038, 0.0052, 0.117,  0.0162, 0.165],
        [0.0057, 0.195,  0.015,  0.009,  0.011],
        [0.0038, 0.0091, 0.0075, 0.027,  0.011],
        [0.0247, 0.0468, 0.0075, 0.117,  0.022]
    ])

    gammas1 = np.array([0.8, 0.65, 0.82, 0.75, 0.7])
    gammas2 = np.array([0.2, 0.1, 0.25, 0.15, 0.2])
    # Using 0 cost to maximize probability
    costs =  np.array([0.15,0.2,0.25,0.1,0.2])#[0.0] * NUM_CELLS)

    # --- 3.2. Create Environments with unique seed for this run ---
    env = SearchEnvVec(p0=prior, gamma1=gammas1, gamma2=gammas2, T=TIME_HORIZON, cost_vector=costs)
    eval_env = SearchEnvVec(p0=prior, gamma1=gammas1, gamma2=gammas2, T=TIME_HORIZON, cost_vector=costs)

    # --- 3.3. Setup Logging and Callbacks for this specific run ---
    run_log_dir = os.path.join(base_log_dir, f"run_{run_id}")
    best_model_dir = os.path.join(run_log_dir, "best_model")
    os.makedirs(run_log_dir, exist_ok=True)
    os.makedirs(best_model_dir, exist_ok=True)

    new_logger = configure(run_log_dir, ["csv", "tensorboard"])

    # Set eval_freq lower to get more frequent evaluation points
    eval_callback = EvalCallback(eval_env,
                                 best_model_save_path=best_model_dir,
                                 log_path=run_log_dir, # Save eval results in run dir
                                 eval_freq=10000, # Evaluate more frequently
                                 n_eval_episodes=100, # Evaluate on 100 episodes during training
                                 deterministic=True,
                                 render=False)

    # --- 3.4. DQN Model Training ---
    model = DQN(
        "MlpPolicy",
        env,
        verbose=0,
        buffer_size=50000,
        learning_starts=1000,
        batch_size=64,
        gamma=0.95, #
        train_freq=(10, "step"),
        gradient_steps=10,
        target_update_interval=1000,
        exploration_fraction=0.2,
        exploration_final_eps=0.01,
        seed=seed,
        device="cpu"
    )

    model.set_logger(new_logger)

    model.learn(total_timesteps=total_timesteps, callback=eval_callback)

    # --- 3.5. Final Evaluation of the Best Model ---
    print(f"Evaluating best model for run {run_id}...")
    best_model_path = os.path.join(best_model_dir, "best_model.zip")
    final_metrics = {}

    if os.path.exists(best_model_path):
        best_model = DQN.load(best_model_path, env=eval_env)
        num_final_eval_episodes = 100
        total_reward = 0
        num_successes = 0
        detection_times = []
        episode_lengths = []

        # Each episode will now have a unique seed based on the run_id and episode index
        for i_episode in range(num_final_eval_episodes):
            episode_seed = seed * num_final_eval_episodes + i_episode # Unique seed for each episode
            obs, info = eval_env.reset(seed=episode_seed)
            done = False
            episode_reward = 0
            episode_len = 0

            while not done:
                action, _ = best_model.predict(obs, deterministic=True)
                obs, reward, terminated, truncated, info = eval_env.step(action)
                done = terminated or truncated
                episode_reward += reward
                episode_len += 1

                if info.get('found_o1'):
                    detection_times.append(episode_len)

            total_reward += episode_reward
            episode_lengths.append(episode_len)
            if info.get('found_o1'):
                 num_successes += 1

        mean_reward = total_reward / num_final_eval_episodes
        success_rate = num_successes / num_final_eval_episodes
        mean_detection_time = np.mean(detection_times) if detection_times else -1
        mean_episode_length = np.mean(episode_lengths)

        final_metrics = {
            'success_rate': success_rate,
            'avg_detection_time': mean_detection_time,
            'avg_episode_reward': mean_reward,
            'avg_episode_length': mean_episode_length
        }


        print(f"Run {run_id} Final Evaluation Results (Best Model):")
        print(f"  Success Rate: {final_metrics['success_rate']:.4f} ({final_metrics['success_rate']*100:.2f}%)")
        print(f"  Average Detection Time (Successful Episodes): {final_metrics['avg_detection_time']:.2f}")
        print(f"  Average Episode Reward: {final_metrics['avg_episode_reward']:.4f}")
        print(f"  Average Episode Length: {final_metrics['avg_episode_length']:.2f}")
    else:
        print(f"No best model found for run {run_id}.")


    print(f"===== COMPLETED DQN RUN {run_id} ====")
    return final_metrics


if __name__ == '__main__':
    # --- Configuration for Multiple Runs ---
    NUM_RUNS = 100 # Set to 100 seeds as requested
    TOTAL_TIMESTEPS = 200_000
    BASE_LOG_DIR = "./dqn_z_vector_logs/"

    # --- Execute All Runs ---
    all_final_success_rates = []
    all_final_detection_times = []
    all_final_rewards = []
    all_final_episode_lengths = []

    total_experiment_start_time = time.time()

    for i in range(NUM_RUNS):
        run_seed = i # Seeds go from 0 to 99
        final_metrics = run_single_experiment(
            run_id=i,
            base_log_dir=BASE_LOG_DIR,
            seed=run_seed,
            total_timesteps=TOTAL_TIMESTEPS
        )

        # Collect metrics from each run
        if final_metrics: # Only collect if evaluation was successful
            all_final_success_rates.append(final_metrics['success_rate'])
            if final_metrics['avg_detection_time'] != -1:
                 all_final_detection_times.append(final_metrics['avg_detection_time'])
            all_final_rewards.append(final_metrics['avg_episode_reward'])
            all_final_episode_lengths.append(final_metrics['avg_episode_length'])


    total_experiment_end_time = time.time()

    # --- Calculate and Display Overall Metrics ---
    print(f"\nAll {NUM_RUNS} DQN training runs are complete.")
    print(f"Log files are saved in '{BASE_LOG_DIR}'.")
    print(f"Total time for all runs: {total_experiment_end_time - total_experiment_start_time:.2f} seconds")

    print("\n--- Overall DQN Policy Results Across All Seeds ---")
    print(f"Average Success Rate: {np.mean(all_final_success_rates):.4f} ({np.mean(all_final_success_rates)*100:.2f}%)")
    print(f"Standard Deviation of Success Rate: {np.std(all_final_success_rates):.4f}")

    if all_final_detection_times:
        print(f"Average Detection Time (Successful Episodes): {np.mean(all_final_detection_times):.2f}")
        print(f"Standard Deviation of Detection Time: {np.std(all_final_detection_times):.2f}")
    else:
        print("Average Detection Time (Successful Episodes): N/A (no successful episodes across all runs)")

    print(f"Average Episode Reward: {np.mean(all_final_rewards):.4f}")
    print(f"Standard Deviation of Episode Reward: {np.std(all_final_rewards):.4f}")

    print(f"Average Episode Length: {np.mean(all_final_episode_lengths):.2f}")
    print(f"Standard Deviation of Episode Length: {np.std(all_final_episode_lengths):.2f}")


2025-11-07 06:09:53.880714: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-11-07 06:09:53.971525: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-11-07 06:09:55.597969: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



===== STARTING DQN RUN 0 with SEED 0 ====


/home/jey/anaconda3/envs/dataanot/lib/python3.10/site-packages/stable_baselines3/common/evaluation.py:70: UserWarning: Evaluation environment is not wrapped with a ``Monitor`` wrapper. This may result in reporting modified episode lengths and rewards, if other wrappers happen to modify these. Consider wrapping environment first with ``Monitor`` wrapper.
  warnings.warn(


Eval num_timesteps=10000, episode_reward=-0.62 +/- 1.02
Episode length: 7.45 +/- 3.36
New best mean reward!
Eval num_timesteps=20000, episode_reward=-0.30 +/- 1.05
Episode length: 6.45 +/- 3.40
New best mean reward!
Eval num_timesteps=30000, episode_reward=-0.46 +/- 1.02
Episode length: 7.16 +/- 2.91
Eval num_timesteps=40000, episode_reward=-0.51 +/- 0.97
Episode length: 7.63 +/- 2.46
Eval num_timesteps=50000, episode_reward=-0.55 +/- 1.04
Episode length: 7.20 +/- 3.19
Eval num_timesteps=60000, episode_reward=-0.66 +/- 0.98
Episode length: 7.98 +/- 2.77
Eval num_timesteps=70000, episode_reward=-0.37 +/- 0.99
Episode length: 6.98 +/- 3.20
Eval num_timesteps=80000, episode_reward=-0.53 +/- 0.97
Episode length: 7.43 +/- 3.14
Eval num_timesteps=90000, episode_reward=-0.38 +/- 1.04
Episode length: 6.71 +/- 3.31
Eval num_timesteps=100000, episode_reward=-0.69 +/- 0.96
Episode length: 8.00 +/- 2.84
Eval num_timesteps=110000, episode_reward=-0.41 +/- 0.91
Episode length: 7.35 +/- 3.07
Eval num

In [5]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# --- Configuration ---
BASE_LOG_DIR = "./dqn_z_vector_logs/" # <-- MUST MATCH THE TRAINING SCRIPT
NUM_RUNS = 10 # <-- MUST MATCH THE TRAINING SCRIPT
ROLLOUT_PLOT_FILENAME = 'dqn_z_vector_mean_rollout_reward_plot.png'
EVAL_PLOT_FILENAME = 'dqn_z_vector_mean_evaluation_reward_plot.png'
EVAL_FREQ = 5000 # <-- Must match the eval_freq in the training script

def process_and_plot_line(list_of_dfs: list, x_col: str, y_col: str, title: str, 
                          xlabel: str, ylabel: str, color: str, output_filename: str):
    """
    Processes a list of dataframes to calculate and plot the mean and std dev
    as a continuous, interpolated line with a shaded region.
    """
    if not list_of_dfs:
        print(f"Warning: No data found for column '{y_col}'. Skipping plot.")
        return

    # 1. Create a common x-axis (timesteps) for all runs using interpolation
    max_x = max(df[x_col].max() for df in list_of_dfs)
    common_x_axis = np.linspace(0, max_x, num=500) # 500 points for a smooth curve
    
    # 2. Interpolate y-values for each run onto the common x-axis
    interpolated_y_values = []
    for df in list_of_dfs:
        interpolated_y = np.interp(common_x_axis, df[x_col], df[y_col])
        interpolated_y_values.append(interpolated_y)
        
    # 3. Calculate mean and standard deviation across all runs
    y_matrix = np.array(interpolated_y_values)
    mean_y = np.mean(y_matrix, axis=0)
    std_y = np.std(y_matrix, axis=0)
    
    # 4. Plotting
    plt.figure(figsize=(12, 7))
    plt.plot(common_x_axis, mean_y, color=color, label='Mean')
    plt.fill_between(common_x_axis, mean_y - std_y, mean_y + std_y, 
                     color=color, alpha=0.25, label='Standard Deviation')
                     
    plt.title(title, fontsize=16)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    
    # 5. Save the plot
    plt.savefig(output_filename)
    print(f"Saved plot to '{output_filename}'")
    plt.close()

def process_and_plot_errorbars(list_of_dfs: list, x_col: str, y_col: str, title: str, 
                               xlabel: str, ylabel: str, color: str, output_filename: str):
    """
    Processes a list of dataframes to calculate and plot the mean and std dev
    as discrete error bar points.
    """
    if not list_of_dfs:
        print(f"Warning: No data found for column '{y_col}'. Skipping plot.")
        return

    # 1. Group rewards by the evaluation timestep
    # We round the timesteps to the nearest EVAL_FREQ to group them
    eval_data = {}
    for df in list_of_dfs:
        for _index, row in df.iterrows():
            # Round the timestep to the nearest evaluation point
            timestep = int(round(row[x_col] / EVAL_FREQ)) * EVAL_FREQ
            if timestep == 0: continue # Skip 0-step evaluation if present
            
            reward = row[y_col]
            if timestep not in eval_data:
                eval_data[timestep] = []
            eval_data[timestep].append(reward)

    if not eval_data:
        print(f"Warning: No evaluation data could be grouped. Skipping plot.")
        return

    # 2. Calculate mean and std dev for each timestep
    timesteps = sorted(eval_data.keys())
    mean_rewards = [np.mean(eval_data[t]) for t in timesteps]
    std_devs = [np.std(eval_data[t]) for t in timesteps]
    
    # 3. Plotting
    plt.figure(figsize=(12, 7))
    
    # Use plt.errorbar to create the plot
    # --- MODIFIED ---
    # fmt='-o' creates a line connecting the points, with markers at each point.
    # capsize=5 adds the small caps to the error bars
    plt.errorbar(timesteps, mean_rewards, yerr=std_devs, fmt='-o', color=color, 
                 capsize=5, label='Mean Evaluation Reward (± 1 Std Dev)',
                 linestyle='-', marker='o', markersize=5) # Explicitly set line and marker
                     
    plt.title(title, fontsize=16)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.legend()
    plt.grid(True, which='both', linestyle='--', linewidth=0.5)
    plt.tight_layout()
    
    # 4. Save the plot
    plt.savefig(output_filename)
    print(f"Saved plot to '{output_filename}'")
    plt.close()


if __name__ == '__main__':
    all_rollout_dfs = []
    all_eval_dfs = []

    # --- Load Data from All Runs ---
    print(f"Loading data from '{BASE_LOG_DIR}'...")
    for i in range(NUM_RUNS):
        file_path = os.path.join(BASE_LOG_DIR, f"run_{i}", "progress.csv")
        try:
            df = pd.read_csv(file_path)
            # Add rollout data if the column exists
            if 'rollout/ep_rew_mean' in df.columns:
                all_rollout_dfs.append(df[['time/total_timesteps', 'rollout/ep_rew_mean']].dropna())
            # Add evaluation data if the column exists
            if 'eval/mean_reward' in df.columns:
                 # We need all timesteps for the error bar plot, not just the interpolated ones
                 all_eval_dfs.append(df[['time/total_timesteps', 'eval/mean_reward']].dropna())
        except FileNotFoundError:
            print(f"Warning: Could not find '{file_path}'. Skipping run {i}.")
        except Exception as e:
            print(f"An error occurred loading data for run {i}: {e}")

    # --- Generate Plots ---
    
    # Plot 1: Mean Training (Rollout) Reward - USES THE LINE PLOT
    process_and_plot_line(
        list_of_dfs=all_rollout_dfs,
        x_col='time/total_timesteps',
        y_col='rollout/ep_rew_mean',
        title='DQN (z-vector) - Mean Training Reward (Rollout) vs. Timesteps',
        xlabel='Total Timesteps',
        ylabel='Mean Episode Reward',
        color='darkorange',
        output_filename=ROLLOUT_PLOT_FILENAME
    )
    
    # Plot 2: Mean Evaluation Reward - USES THE NEW ERROR BAR PLOT
    process_and_plot_errorbars(
        list_of_dfs=all_eval_dfs,
        x_col='time/total_timesteps',
        y_col='eval/mean_reward',
        title='DQN (z-vector) - Mean Evaluation Reward vs. Timesteps',
        xlabel='Total Timesteps',
        ylabel='Mean Episode Reward',
        color='darkred',
        output_filename=EVAL_PLOT_FILENAME
    )

    print("\nPlotting complete.")

Loading data from './dqn_z_vector_logs/'...
Saved plot to 'dqn_z_vector_mean_rollout_reward_plot.png'


/tmp/ipykernel_865939/617927569.py:96: UserWarning: linestyle is redundantly defined by the 'linestyle' keyword argument and the fmt string "-o" (-> linestyle='-'). The keyword argument will take precedence.
  plt.errorbar(timesteps, mean_rewards, yerr=std_devs, fmt='-o', color=color,
/tmp/ipykernel_865939/617927569.py:96: UserWarning: marker is redundantly defined by the 'marker' keyword argument and the fmt string "-o" (-> marker='o'). The keyword argument will take precedence.
  plt.errorbar(timesteps, mean_rewards, yerr=std_devs, fmt='-o', color=color,


Saved plot to 'dqn_z_vector_mean_evaluation_reward_plot.png'

Plotting complete.
